Grab fablib manager

In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import shlex
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,sjzhu_0000105651
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Find a site

In [8]:
# find a site with appropriate cores
cores_per_node = 8
ram_per_node = 32
disk_per_node = 20 # we will modify this later to give fileservers more
site1 = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 3 and x["ram_available"] > ram_per_node * 3 and x["disk_available"] > disk_per_node * 3
)
fablib.show_site(site1, 'json')

{
    "name": "UCSD",
    "state": "Active",
    "address": "10100 Hopkins Drive,CA 92093",
    "location": [
        32.88868022489132,
        -117.23932400094392
    ],
    "hosts": 5,
    "cpus": 10,
    "cores_available": 318,
    "cores_capacity": 640,
    "cores_allocated": 322,
    "ram_available": 1630,
    "ram_capacity": 2510,
    "ram_allocated": 880,
    "disk_available": 54537,
    "disk_capacity": 58717,
    "disk_allocated": 4180,
    "nic_basic_available": 583,
    "nic_basic_capacity": 635,
    "nic_basic_allocated": 52,
    "nic_connectx_6_available": 2,
    "nic_connectx_6_capacity": 2,
    "nic_connectx_6_allocated": 0,
    "nic_connectx_5_available": 4,
    "nic_connectx_5_capacity": 4,
    "nic_connectx_5_allocated": 0,
    "nvme_available": 15,
    "nvme_capacity": 16,
    "nvme_allocated": 1,
    "tesla_t4_available": 4,
    "tesla_t4_capacity": 4,
    "tesla_t4_allocated": 0,
    "rtx6000_available": 5,
    "rtx6000_capacity": 6,
    "rtx6000_allocated": 1,
  

'{\n    "name": "UCSD",\n    "state": "Active",\n    "address": "10100 Hopkins Drive,CA 92093",\n    "location": [\n        32.88868022489132,\n        -117.23932400094392\n    ],\n    "hosts": 5,\n    "cpus": 10,\n    "cores_available": 318,\n    "cores_capacity": 640,\n    "cores_allocated": 322,\n    "ram_available": 1630,\n    "ram_capacity": 2510,\n    "ram_allocated": 880,\n    "disk_available": 54537,\n    "disk_capacity": 58717,\n    "disk_allocated": 4180,\n    "nic_basic_available": 583,\n    "nic_basic_capacity": 635,\n    "nic_basic_allocated": 52,\n    "nic_connectx_6_available": 2,\n    "nic_connectx_6_capacity": 2,\n    "nic_connectx_6_allocated": 0,\n    "nic_connectx_5_available": 4,\n    "nic_connectx_5_capacity": 4,\n    "nic_connectx_5_allocated": 0,\n    "nvme_available": 15,\n    "nvme_capacity": 16,\n    "nvme_allocated": 1,\n    "tesla_t4_available": 4,\n    "tesla_t4_capacity": 4,\n    "tesla_t4_allocated": 0,\n    "rtx6000_available": 5,\n    "rtx6000_capacity

In [9]:
slice_name = 'ndn-poc-sjzhu2'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

hosts = [f"{site1.lower()}-c1", f"{site1.lower()}-r1", f"{site1.lower()}-p1"]

In [4]:
# Define a subnet to use for the L2 Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [11]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

for h in hosts:

    node = slice.add_node(name=f"{h}-a", cores=cores_per_node, ram=ram_per_node, disk=disk_per_node, site=site1, image='default_ubuntu_22')
    node.add_fabnet()
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")

#Submit Slice Request
slice.submit();


Retry: 8, Time: 861 sec


ID,2cad5889-528a-41cc-a84d-a381a165b6e2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-08 05:21:27 +0000
Lease Start (UTC),2023-11-07 05:21:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
c879caec-18e8-4d36-9dc0-d8adb355e9a8,ucsd-c1-a,8,32,100,default_ubuntu_22,qcow2,ucsd-w5.fabric-testbed.net,UCSD,ubuntu,132.249.252.174,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.174,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0598a0ee-8dd9-4e70-b205-f8e9e996871a,ucsd-p1-a,8,32,100,default_ubuntu_22,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.183,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.183,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2fb6f21d-886e-4d2d-acd5-3dc471a305df,ucsd-r1-a,8,32,100,default_ubuntu_22,qcow2,ucsd-w5.fabric-testbed.net,UCSD,ubuntu,132.249.252.152,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.152,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
d736817d-37ee-465a-b6cf-0e80388c047e,FABNET_IPv4_UCSD,L3,FABNetv4,UCSD,10.134.139.0/24,10.134.139.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
ucsd-c1-a-FABNET_IPv4_UCSD_nic-p1,p1,ucsd-c1-a,FABNET_IPv4_UCSD,100,auto,,2E:C0:02:B3:AB:52,enp6s0,enp6s0,10.134.139.4,4
ucsd-r1-a-FABNET_IPv4_UCSD_nic-p1,p1,ucsd-r1-a,FABNET_IPv4_UCSD,100,auto,,2A:E9:CB:CD:B7:2C,enp6s0,enp6s0,10.134.139.2,4
ucsd-p1-a-FABNET_IPv4_UCSD_nic-p1,p1,ucsd-p1-a,FABNET_IPv4_UCSD,100,auto,,02:DB:9A:28:A0:C2,enp6s0,enp6s0,10.134.139.3,4



Time to print interfaces 862 seconds


Extend the slice

In [12]:
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}
FS_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 4*1024},
        'filePrefix': 'producer',
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_ARGS['socketName'],
        'id': MEMIF_ARGS['id'],
        'dataroom': 9000,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': "/producer/usr-local-share", 'path': "/usr/local/share"}
        ],
        'segmentLen': 6 * 1024,
    },
}

adj_list = {
    hosts[0]: [hosts[1]],
    hosts[1]: [hosts[0], hosts[2]],
    hosts[2]: [hosts[1]],
}

forwarder_hashes = {}
face_hashes = {}

In [23]:
# add eth ports to each node
for h in hosts:
    node = slice.get_node(f"{h}-a")
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif enp1s0 --xdp --mtu 3000
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(f"{h}-a")
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

true
{"id":"J01GPGAEL8SU0","isDown":false,"macAddr":"fa:16:3e:03:c3:95","mtu":3000,"name":"net_af_xdp_2","numaSocket":null}
true
{"id":"U3K8CT4UT8CPG","isDown":false,"macAddr":"fa:16:3e:1f:6f:c5","mtu":3000,"name":"net_af_xdp_2","numaSocket":null}
true
{"id":"1J5MLBV71DVGC","isDown":false,"macAddr":"fa:16:3e:a8:cc:1c","mtu":3000,"name":"net_af_xdp_2","numaSocket":null}
{"id":"JCB0FO0HTSPE8BHM"}
{"id":"UFUOQLE1LK89IOLM"}
{"id":"UFUOQLE1L4APMODJ"}
{"id":"1VF633LO95UG1CUK"}


In [22]:
print(forwarder_hashes)
print(face_hashes)
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
print(f"{hosts[0]}:{hosts[1]}")

{'ucsd-c1': {'id': 'MHPHERKK3OAHS', 'isDown': False, 'macAddr': 'fa:16:3e:03:c3:95', 'mtu': 3000, 'name': 'net_af_xdp_2', 'numaSocket': None}, 'ucsd-r1': {'id': 'U5MKUIM1T316O', 'isDown': False, 'macAddr': 'fa:16:3e:1f:6f:c5', 'mtu': 3000, 'name': 'net_af_xdp_2', 'numaSocket': None}, 'ucsd-p1': {'id': 'LKRH5QRH24344', 'isDown': False, 'macAddr': 'fa:16:3e:a8:cc:1c', 'mtu': 3000, 'name': 'net_af_xdp_2', 'numaSocket': None}}
{'ucsd-r1:ucsd-c1': {'id': 'MTJ1OJUBBGEHRGC9'}, 'ucsd-c1:ucsd-r1': {'id': 'U9S48QSULN76LF9H'}, 'ucsd-p1:ucsd-r1': {'id': 'U9S48QSULF46RFPV'}, 'ucsd-r1:ucsd-p1': {'id': 'LOH1JIHEAG64FQNG'}}
ucsd-c1:ucsd-r1


In [24]:
# add fileserver configuration to producer
producer_name = hosts[2] # hardcode this for now
producer_node = slice.get_node(f"{producer_name}-a")
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')
# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

{"id":"1VB61BNC14RKVF7CD3JCB8CRN4G5CK8"}
true


In [25]:
# add FIB entries to point to producer
c_node = slice.get_node(f"{hosts[0]}-a")
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(f"{hosts[1]}-a")
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

{"id":"JCF0DG25L1OQI9G82R8A6J1IRUCRVF0"}
{"id":"UFQOOTCLT18T2QS9C6T4DD0B431AMK8"}


('{"id":"UFQOOTCLT18T2QS9C6T4DD0B431AMK8"}\n', '')

In [26]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [27]:
slice.update()

In [28]:
slice.show()

ID,2cad5889-528a-41cc-a84d-a381a165b6e2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-11 07:11:53 +0000
Lease Start (UTC),2023-11-07 05:21:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,2cad5889-528a-41cc-a84d-a381a165b6e2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-11 07:11:53 +0000
Lease Start (UTC),2023-11-07 05:21:28 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


TODO:
- script forwarder setup
  - use example activation parameters, excluding the CPU cores
- script forwarders to connect to each other
  - will need mac address from each and then create ether-face between pairs
- script fileserver setup
  - use example activation parameters, possibly editing the mountpoints
- use fileserver docs to setup FIB to fileserver using /producerN name
- create FIB entries for fileservers in each forwarder